## Web Scraping Trading View Screener

#### Limitations: Only can get the base data from the page, not posible to get the others interior tabs or  buttons than the overview

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Diccionario de screeners y sus respectivas partes de URL
screeners = {
    "all_index": "markets/indices/quotes-all/",
    "major_index": "markets/indices/quotes-major/",
    "us_index": "markets/indices/quotes-us/",
    "snp_index": "markets/indices/quotes-snp/",
    "currency_index": "markets/indices/quotes-currency/",
    "all_stocks_US": "markets/stocks-usa/market-movers-all-stocks/"
}
# Selección del screener
screener_selected = "all_stocks_US"
base_url = "https://en.tradingview.com/"
url = base_url + screeners[screener_selected]


In [4]:
def get_response_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        print("La petición ha sido exitosa")
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find('table')
        if not table:
            raise ValueError("No se encontró la tabla en la página")
        index = []
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols:
                ticker = cols[0].find('a').text
                name = cols[0].find('sup').text
                other_data = [td.text for td in cols[1:]]
                index.append([ticker, name] + other_data)
            #index.append([td.text for td in row.find_all('td')])

        head = table.find_all('tr')[0].find_all('th')
        head = [th.text for th in head]
        head = ['Ticker', 'Nombre'] + head[1:]
        data= pd.DataFrame(index, columns=head)
        return data
    else: 
        print("Error en la petición")
    

In [86]:
data= get_response_data(response)
data.head()

,Ticker,Nombre,Price,Change %,Volume,Rel Volume,Market cap,P/E,EPS dilTTM,EPS dil growthTTM YoY,Div yield %TTM,Sector,Analyst Rating
0,A,"Agilent Technologies, Inc.",137.32 USD,+1.73%,1.718 M,1.06,40.065 B USD,32.45,4.23 USD,−6.72%,0.67%,Commercial services,Neutral
1,AA,Alcoa Corporation,33.43 USD,+1.58%,5.799 M,0.86,6.003 B USD,—,−3.08 USD,+62.20%,1.20%,Non-energy minerals,Buy
2,AACI,Armada Acquisition Corp. I,9.55 USD,−15.93%,19.115 K,0.28,68.065 M USD,—,−0.20 USD,−523.34%,0.00%,Finance,—
3,AACT,Ares Acquisition Corporation II,10.72 USD,−0.05%,232.667 K,1.20,536 M USD,28.75,0.37 USD,—,0.00%,Finance,—
4,AADI,"Aadi Bioscience, Inc.",1.58 USD,−1.86%,139.132 K,1.15,38.796 M USD,—,−2.55 USD,+3.11%,0.00%,Health technology,Strong buy


## More data

### NOT WORKING

In [8]:
# Configuración del WebDriver
driver_path = r"chromedriver-win32\chromedriver.exe" 

In [68]:
# Trading view api
api = "https://scanner.tradingview.com/cfd/scan"

In [69]:
# Headers
headers = {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "es-419,es;q=0.9",
    "content-type": "text/plain;charset=UTF-8",
    "origin": "https://www.tradingview.com",
    "referer": "https://www.tradingview.com/",
    "sec-ch-ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
}

In [70]:
# Payload de la petición (esto es un ejemplo, ajusta según sea necesario)
payload = {
    "filter": [],
    "options": {
        "lang": "en"
    },
    "markets": [
        "cfd"
    ],
    "symbols": {
        "query": {
            "types": []
        },
        "tickers": []
    },
    "columns": [
        "logoid",
        "name",
        "close",
        "change",
        "change_abs",
        "high",
        "low",
        "volume",
        "recommendation",
        "description"
    ],
    "sort": {
        "sortBy": "name",
        "sortOrder": "asc"
    },
    "range": [
        0,
        100
    ]
}

In [71]:
response = requests.post(url, headers=headers, json=payload)
data = response.json()

In [74]:
data

{'page_title': 'Major Indices — Quotes and Overview — TradingView',
 'page_title_suffix': '',
 'wide_page': True,
 'division': None,
 'show_ads': None,
 'pagination': None,
 'canonical': {'is_canonical': True,
  'url': 'https://www.tradingview.com/markets/indices/quotes-major/'},
 'additional_meta': {'robots': {'index': True, 'follow': True},
  'open_graph': {'tags': [{'name': 'meta',
     'attributes': [{'name': 'property', 'value': 'og:site_name'},
      {'name': 'content', 'value': 'TradingView'}],
     'pair': False,
     'content': None},
    {'name': 'meta',
     'attributes': [{'name': 'property', 'value': 'og:type'},
      {'name': 'content', 'value': 'website'}],
     'pair': False,
     'content': None}]}},
 'show_big_footer': True,
 'seo_locale_suffix': '',
 'meta_description': 'See major world indices on a cutting-edge financial platform. Check out the latest price updates and sort stock market indices using various metrics.',
 'show_hreflang_tags': True,
 'path_wo_paginati

In [91]:
# Procesar los datos de la respuesta
df = pd.DataFrame()
if "data" in data:
    rows = []
    for item in data["data"]:
        symbol = item["s"]
        details = item["d"]
        rows.append([symbol] + details)
    
    columns = ["Symbol", "Ticker", "Name", "URL", "Type", "Category", "Change %", "Change Abs", "High", "Low", "Volume", "Tech Rating"]
    df = pd.DataFrame(rows, columns=columns)

print(df)

Empty DataFrame
Columns: []
Index: []


#### Using Trading View API

In [92]:
def get_tradingview_market_data(market="america", columns=None):
    url = f"https://scanner.tradingview.com/{market}/scan"
    payload = {
        "filter": [],
        "options": {"lang": "es"},
        "markets": ["america"],
        "symbols": {"query": {"types": []}, "tickers": []},
        "columns": columns if columns else ["Recommend.Other", "volume", "change", "close", "description"],
        "sort": {"sortBy": "volume", "sortOrder": "desc"},
        "range": [0, 150]
    }
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.post(url, json=payload, headers=headers)
    data = response.json()
    
    # Convertir los datos a un DataFrame
    df = pd.DataFrame(data['data'])
    df.columns = columns if columns else ["Recommend.Other", "volume", "change", "close", "description"]
    
    return df

In [94]:
df = get_tradingview_market_data()
print(df.head())

ValueError: Length mismatch: Expected axis has 2 elements, new values have 5 elements